In [1]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
num_classes = 6
img_rows, img_cols = 48, 48
batch_size = 16

train_data_dir = 'fer2013/train'
validation_data_dir = 'fer2013/validation'

# Creating Train Data and Validation Data

In [3]:
img_train_gen=ImageDataGenerator(rescale=1.0/255,\
                                  rotation_range=30,\
                                  shear_range=0.3,\
                                  zoom_range=0.3,\
                                  width_shift_range=0.4,\
                                  height_shift_range=0.4,\
                                  horizontal_flip=True,\
                                  fill_mode='nearest')
img_valid_gen = ImageDataGenerator(rescale=1./255)

train_generator = img_train_gen.flow_from_directory(
        train_data_dir,
        color_mode = 'grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
validation_generator = img_valid_gen.flow_from_directory(
        validation_data_dir,
        color_mode = 'grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

Found 28273 images belonging to 6 classes.
Found 3534 images belonging to 6 classes.


# Keras VGG Model

In [4]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding = 'same', kernel_initializer="he_normal",
                 input_shape = (img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding = "same", kernel_initializer="he_normal", 
                 input_shape = (img_rows, img_cols, 1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #2: second CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #3: third CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #4: third CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #5: first set of FC => RELU layers
model.add(Flatten())
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block #6: second set of FC => RELU layers
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block #7: softmax classifier
model.add(Dense(num_classes, kernel_initializer="he_normal"))
model.add(Activation("softmax"))

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2

In [5]:
checkpoint=ModelCheckpoint(r'C:\Users\Mohit\Documents\Interested ML Projects\Emotion Detector DL\emotion_little_vgg_3.h5')
'''earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)'''

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [ checkpoint, reduce_lr]#earlystop,

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics = ['accuracy'])

nb_train_samples = 28273
nb_validation_samples = 3534
epochs = 50

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
1767/1767 [==============================] - 204s 115ms/step - loss: 1.9581 - acc: 0.2097 - val_loss: 1.7403 - val_acc: 0.2514- loss: 2.7352 - ETA: 3:32 - lo - ETA: 3:12 - loss: 2.6651 - ETA: 3:04 - loss: 2.6243 - acc: - ETA: 3:00 - loss: - ETA: 2:51 - loss: 2.5928 - acc: 0.17 - ETA: 2:50 - loss: 2.588 - ETA: 2:45 - loss: 2.5736 - a - ETA: 2:42 - loss: 2.5638 - acc - - ETA: 2:16 - loss: 2 - ETA: 2:13 - loss: 2.4171 - acc: 0.184 - ETA: 2:13 - loss: 2.4166 - acc: 0 - ETA: 2:12 - loss: 2 - ETA: 2:10 - loss: 2.3959 - acc: 0.1 - ETA: 2:09 - loss: 2.3913 - acc: 0.184 - ETA: 2:09 - loss: 2.3902 - ETA: 2:08 - loss: 2.3716 - ETA: 2:06 - loss: 2.3600 - acc: 0.18 - ETA: 2:06 - loss: 2.3597 - acc: 0.18 - ETA: 2:01 - loss: 2.3280 - ETA: 1:59 - loss: 2.3163 -  - ETA: 1:58 - loss:  - ETA: 1:5 - ETA: 1:52 - loss: 2.2780 - acc: 0.1 - ETA: 1:5 - ETA: 1:49 - loss: 2.26 - ETA: 1:47 - loss: 2.2524 - acc: 0. - ETA: 1:47 - loss: 2.2 - ETA: 1:45 - los

1767/1767 [==============================] - 151s 86ms/step - loss: 1.4156 - acc: 0.4365 - val_loss: 1.4589 - val_acc: 0.4650A: 2:04 - los - ETA: 2:06 - loss:  - ETA: 2:07 - los - ETA: 2:06 - ETA: 2:05 - loss:  - ETA: 2:04 - loss: 1.4405 - acc: 0. - ETA: 2:04 - los - - ETA: 1:40 - loss: 1.4230 - acc: 0.43 - ETA: 1:40 - lo - ETA: 1:38 - loss: 1.4251 -  - ETA: 1:37 - lo
Epoch 10/50
1767/1767 [==============================] - 149s 84ms/step - loss: 1.3892 - acc: 0.4505 - val_loss: 1.4727 - val_acc: 0.4517A: 2:04 - loss: - ETA: 2:04 - loss: 1.3998 - - ETA: 2:03 - loss: 1.4001 - acc:  - ETA: 2:03 - loss: 1.3976 - acc:  - ETA: 2:00 - loss: 1.3973 - acc: - ETA: 2:00 - loss: 1.3933 - acc: 0.45 - - ETA: 1:58 - loss: 1.3877 - ETA: 1:57 - loss: 1.3864 - acc - ETA: 1:57 - loss:  - ETA: 6s - loss: 1.3907 - a - ETA: 5s - loss: 1.3905 - acc: 0. - ETA: 5s - loss: 1.3904  - ETA: 3s - ETA: 1s - loss: 1.3881 - acc - ETA: 0s - loss: 1.3884 - a
Epoch 11/50
1767/1767 [==============================] - 150s

1767/1767 [==============================] - 149s 84ms/step - loss: 1.2697 - acc: 0.5081 - val_loss: 1.3623 - val_acc: 0.5190 - ETA: 2:00 - loss: 1.2899 - a - ETA: 2:00 - loss: 1.2868  - ETA: 1:59 - l - ETA: 1:58 - loss: 1.2930 - acc: 0.503 - ETA: 1:55 - loss: 1.2873 - acc: 0. - ETA: 1:54  - ETA: 1:53 - loss: 1.2822 - ETA: 1:52 - loss: 1.2857 - - ETA: 1:51 - loss: 1.28 - ETA: 1:49 - loss: 1.2891 - ETA: 1:48 - loss: 1.28 - ETA: 1:47 -  - ETA: 1:45 - loss: 1. - ETA: 1:44 - loss: 1.2860 - ac - ETA: 1:43 - loss: 1.2851 - acc: - ETA: 1:43 - loss: 1.2852 - a - ETA: 1:42 - loss: 1.28 - ETA: 1:41 - loss: 1.28 - ETA: 1:39 - loss: 1.2873 - acc: 0.503 - ETA: 1:39 - loss: 1.2871 -  - ETA: 1:35 - loss: 1.2856 - acc: 0. - ETA: 1: - ETA: 1:29 - loss: - ETA: 1:27 - loss: 1. - ETA: 1:26 - loss - ETA: 1:24 - loss: 1.2794 - acc: 0. - ETA: 1:24  - ETA: 1:22  - ETA: 1:16 - loss: 1.2799 - acc: 0.506 - ETA: 1:16 - loss: 1.2795 - acc: 0.50 - ETA: 1:16 - loss: 1.2796 - acc: - ETA: 25s -  - ETA: 24s - loss: 1.2

1767/1767 [==============================] - 148s 84ms/step - loss: 1.2365 - acc: 0.5203 - val_loss: 1.3797 - val_acc: 0.52132:01 - loss: 1.1716  - ETA:  - ET - ETA: 2:04 - loss: 1. - ETA: 2:04 - loss: 1.2129 - a - ETA: 2:03 - loss: 1.2065 - ETA: 2:01 - loss - ETA: 1:59 - loss: 1.2191 - acc: 0 - ETA: 1:59 - loss: 1.2176 - acc - ETA: 1:59 - loss: 1.2213 - acc: 0. - ETA: 1:59 - loss - ET - ETA: 1:52 - loss: 1.2251 - acc: 0.525 - E - E - ETA: 1:46 - loss: 1.2184 - acc: 0.5 - ETA: 1:46 - loss: 1.2181 - acc - ETA - ETA: 1:36 - loss: 1.22 - ETA: 1:35 - loss:  - ETA: 1:30 - lo - ETA: 1:28 - loss: 1.2319 - acc: 0. - ETA: 1:28 - loss: 1.2 - ETA: 1:26 - loss: 1.2303 - acc: 0.524 - ETA: 1:26 - loss: 1.2308 - acc: 0. - ETA: 1:26 - loss: 1 - ETA: 1:24 - loss: 1.2335 - acc:  - ETA: 1:21 - loss: 1. - ETA: 1:19 - loss: - ETA: 1:17 - loss: - ETA:  - E - ETA: 9s - loss: 1.2367 - acc: 0.51 - ETA: 8s - loss: 1.2368 - acc: 0 - ETA: 8s - loss: 1.2366 - acc: 0.518  - ETA: 5s - loss: 1.2369 - acc:  - ETA: 4s 

1767/1767 [==============================] - 149s 84ms/step - loss: 1.2307 - acc: 0.5229 - val_loss: 1.3733 - val_acc: 0.5230 1:49 - loss: 1.2868 - acc: 0.494 - ETA: 1:50 - loss: 1.2853 - a - E - ETA: 2:04 - loss: 1.2297 - acc: 0. - ETA: 2:04 - loss: 1.2269 - acc: 0. - ETA: 2:04 - loss: 1.2234 - - ETA: 2:04 - loss: 1.2 - ETA: 2:03 - loss: 1.2265 - a - ETA: 1:57 - loss: 1.2259 - acc: 0.524 - ETA: 1:57 - loss: 1.2257 - ETA: 1:56 - loss: 1.  - E - ETA: 1:49 - loss: 1.2337 - - ETA: 1:48 - loss: 1. - ETA: 1:47 - loss: 1.2366 - acc: 0. - ETA: 1:44 - loss: 1.2382 - a - ETA: 1:43 - loss: 1.2391 - ETA: 1:42 - loss: 1.2 - ETA: 1:40 - loss: 1.2384 - a - ETA: 1:40 - loss: 1.236 - ETA: 1:29 - loss: 1.  - ETA: 1:21 - loss: 1.2305 - acc - ETA: 1:20 - loss: 1.2298 - a - ETA: 1:19 - los - - ETA: 32s - loss: 1.2271 - acc: 0.52 - ETA: 32s -  - E - ETA - ETA: 19s - loss: 1.2273 - acc: 0.52 - ETA: 19s - loss: 1.227 - ETA - ETA: 4s - loss:  - ETA: 2s - loss: 1.2301 - acc: 0.522 - ET
Epoch 33/50
1767/1767 [=

1767/1767 [==============================] - 148s 84ms/step - loss: 1.2372 - acc: 0.5220 - val_loss: 1.3313 - val_acc: 0.5358 - ETA: 2:02 - loss: 1.2344 - acc: 0. - ETA: - ETA: 2:02 - loss: 1.2430 - ETA: - ETA: 1:59 - loss: 1.2378 - - ETA: 1:59 - loss - ETA: 1:57 - loss: 1.2446 - acc: 0 - ETA: 1:57 - loss: 1.2480 -  - ETA: 1:48 - loss: 1.2495 - acc: 0 - ETA: 1:48 - loss: 1 - ETA: 1:46 - loss: 1.2477 -  - ETA: 1:45 - loss: 1.2503 - ac - ETA: 1:45 - loss: 1.2496 - acc - ETA: 1:44 - loss: 1.2501 -  - ETA: 10s - loss: 1.2371 - ETA: 9s - loss:  - ETA: 7s - - ETA: 5s - loss: 1.2368 - ETA: 4s - loss: 1.2375 - ETA:  - ETA: 0s - loss: 1.2371 - acc
Epoch 42/50
1767/1767 [==============================] - 149s 84ms/step - loss: 1.2321 - acc: 0.5276 - val_loss: 1.3930 - val_acc: 0.5244s: 1.2106 - ac - ETA: 1:58 - loss: 1.2297 - acc - - ETA: 2:07 - loss: 1.2187  - ETA: 2:06 - loss: 1.2325 - acc: - ETA: 2:05 - loss: 1.2401 - ac - ETA: 2:05 - loss: 1 - ETA: 2:04 - loss: 1.2346 - ETA: 1:59 - loss: 1.2

1767/1767 [==============================] - 148s 84ms/step - loss: 1.2321 - acc: 0.5262 - val_loss: 1.3453 - val_acc: 0.5307ETA: 1:41 - loss: 1.2308 - acc: - ETA: 1:41 - loss: 1.2307  - ETA: 1:40 - loss: 1. - ETA: 7s - loss: 1.230
Epoch 50/50
1767/1767 [==============================] - 149s 84ms/step - loss: 1.2357 - acc: 0.5248 - val_loss: 1.3796 - val_acc: 0.5259- ETA: 1:58 - ETA: 2:02 - loss: 1.2476 - ac - ETA: 2:03 -  - ETA: 2:04 - ETA: 2:03 - - ETA - ETA: 2:00 - ETA: 1:58 - loss: 1.2370 - acc: 0.52 - ETA: 1:57 - loss: 1.237 - ETA: 1:56 - loss: 1 - ETA: 1:55 - - ETA: 1:51 - loss: 1.2404 - acc - ETA: 1:50 - loss: 1. - ETA: 1: - ETA: 1:46 - loss: 1.2409 - ETA: 1:45 - loss: 1 - ETA: 1:31 - loss: 1.237 - ETA: 1:30 - loss: 1.2365 - acc: 0 - ETA: 1:29 - loss: 1.2361 - acc: 0.522 - ETA: 1:29 - loss: 1.23 - ETA: 1:28 - loss: 1.2360 - acc: 0.52 - ETA: 1:28 - loss: 1.2368 - a - ETA: 1:2 - ETA: 1:25 - loss: 1.2383  - ETA: 1:24 - loss: 1.2411 -  - ETA: 1:23 - loss: 1.2416 - acc: 0.52 - ETA: 

# Testing that with Video Interface using OpenCV

In [ ]:
from keras.models import load_model
classifier = load_model('/home/deeplearningcv/DeepLearningCV/Trained Models/emotion_little_vgg_3.h5')